In [1]:
import pandas as pd
import fasttext
import random
import re
import preprocessor as p
import emoji
import codecs
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
stopwords = set(stopwords.words('english')) - set(['no', 'not'])

In [2]:
contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so is",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have",
}

In [3]:
dataset = pd.read_csv('sentiment.csv')

In [4]:
dataset["Sentiment"].unique()

array([ 1, -1,  0], dtype=int64)

In [5]:
def model_train(dataframe):
    one_text = []
    mone_text = []
    zero_text = []
    for i, v in enumerate(dataframe.loc[:, "Sentiment"].values):
        if v == 1:
            one_text.append("__label__positive " + dataframe.loc[i, "Text"] + "\n")
        elif v == -1:
            mone_text.append("__label__negative " + dataframe.loc[i, "Text"] + "\n")
        else:
            zero_text.append("__label__neutral " + dataframe.loc[i, "Text"] + "\n")
    training_txt = one_text[:2394] + mone_text[:2394] + zero_text[:2394]
    random.shuffle(training_txt)
    training_txt = ''.join(training_txt)
    val_txt = one_text[2394:3192] + mone_text[2394:3192] + zero_text[2394:3192]
    random.shuffle(val_txt)
    val_txt = ''.join(val_txt)
    test_txt = one_text[3192:] + mone_text[3192:] + zero_text[3192:]
    random.shuffle(test_txt)
    test_txt = ''.join(test_txt)
    with open('sentiment.train', 'w', encoding='utf-8') as file:
        file.write(training_txt)

    with open('sentiment.valid', 'w', encoding='utf-8') as file:
        file.write(val_txt)

    with open('sentiment.test', 'w', encoding='utf-8') as file:
        file.write(test_txt)
    model = fasttext.train_supervised(input='sentiment.train', autotuneValidationFile='sentiment.valid', autotuneDuration=60)
    result = model.test('sentiment.test')
    return model, result

In [6]:
model, result = model_train(dataset)
result

(2394, 0.7786131996658312, 0.7786131996658312)

In [7]:
dataset = pd.read_csv('sentiment.csv')
for i, v in enumerate(dataset.loc[:, "Text"].values):
    v = v.lower()
    dataset.loc[i, "Text"] = v
model, result = model_train(dataset)
result

(2394, 0.7393483709273183, 0.7393483709273183)

In [8]:
dataset = pd.read_csv('sentiment.csv')
for i, v in enumerate(dataset.loc[:, "Text"].values):
    v = v.replace('#', ' ')
    v = v.replace('@', ' ')
    v = p.clean(v)
    dataset.loc[i, "Text"] = v
model, result = model_train(dataset)
result

(2394, 0.7652464494569757, 0.7652464494569757)

In [9]:
dataset = pd.read_csv('sentiment.csv')
for i, v in enumerate(dataset.loc[:, "Text"].values):
    v = v.replace("’", "'")
    for key in contractions:
        if key in v:
            v = re.sub(key, contractions[key], v)
    dataset.loc[i, "Text"] = v
model, result = model_train(dataset)
result

(2394, 0.7827903091060986, 0.7827903091060986)

In [10]:
dataset = pd.read_csv('sentiment.csv')
for i, v in enumerate(dataset.loc[:, "Text"].values):
    v = codecs.decode(v, 'unicode_escape')
    v = emoji.demojize(v.encode('utf-16', 'surrogatepass').decode('utf-16'))
    dataset.loc[i, "Text"] = v
model, result = model_train(dataset)
result

c:\users\pooja\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: invalid escape sequence '\ '
  This is separate from the ipykernel package so we can avoid doing imports until


(2394, 0.7844611528822055, 0.7844611528822055)

In [11]:
dataset = pd.read_csv('sentiment.csv')
for i, v in enumerate(dataset.loc[:, "Text"].values):
    v = re.sub("[^a-z0-9 ]", " ", v).strip()
    v = re.sub(" +", " ", v)
    dataset.loc[i, "Text"] = v
model, result = model_train(dataset)
result

(2394, 0.7472848788638262, 0.7472848788638262)

In [12]:
dataset = pd.read_csv('sentiment.csv')
for i, v in enumerate(dataset.loc[:, "Text"].values):
    temp_v = v.split()
    l = WordNetLemmatizer()
    v = [l.lemmatize(word) for word in temp_v if not word in stopwords]
    v = ' '.join(v)
    dataset.loc[i, "Text"] = v
model, result = model_train(dataset)
result

(2394, 0.7560568086883876, 0.7560568086883876)